<div style="text-align:center; margin-top: 1cm;">
    <h2>Київський політехнічний інститут імені Ігоря Сікорського</h2>
    <h2>Приладобудівний факультет</h2>
    <h3>Кафедра автоматизації та систем неруйнівного контролю</h3>
    <br><br><br>
</div>

<div style="text-align:center; margin-top: 5cm;">
    <h2>Практична робота №3 </h2>
    <h2>з предмету "Комп'ютерний зір"</h2>
</div>

<div style="text-align:right; margin-top: 5cm;">
<p>Студент: Погорєлов Богдан<br>
    Група: ПК-51мп<br>
</p>
</div>
<div style="text-align:center; margin-top: 5cm;">
2025 рік  <br><br><br><br>
</div>

Розробити програму, яка виконує такі дії з ЦЗ:
1. Вхідні дані - самостійно отримана цифрова фотографія - 9 об’єктів простої геометричної форми.
2. Програма розпізнає об’єкти на ЦЗ використовуючи аналіз контурів об’єктів. Програма повинна застосувати
мінімум одну геометричну ознаку, яку самостійно визначає розробник програми. У вихідному ЦЗ програма
повинна відобразити контури об’єктів, описані прямокутники та текстовий рядок з інформацією про об’єкт.